In [35]:
import requests
from bs4 import BeautifulSoup

In [36]:
url = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001'
yes24 = requests.get(url)

In [37]:
bs = BeautifulSoup(yes24.text)
titlelist = []
authorlist = []
publisherlist = []
pricelist = []

In [38]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    title = bs.find('ol', class_ = '') \
            .find('li', class_ = 'num' + idx) \
            .find_all('p')[2].text
    title = title.replace('[도서] ', '')
    titlelist.append(title.strip())

In [39]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    author = bs.find('ol', class_ = '') \
                .find('li', class_ = 'num' + idx) \
                .find('p', class_ = 'aupu').text
    author = author.split('|')[0].strip()
    authorlist.append(author)

In [40]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    price = bs.find('ol', class_ = '') \
            .find('li', class_ = 'num' + idx) \
            .find('p', class_ = 'price').text
    price = price.split('원')[0]
    pricelist.append(price)

In [41]:
for i in range(1, 21):
    idx = None
    if(i == 19 or i == 20) : idx = str(i) + '_line'
    else : idx = str(i)
    publisher = bs.find('ol', class_ = '') \
            .find('li', class_ = 'num' + idx) \
            .find('p', class_ = 'aupu').text
    publisher = publisher.split('|')[1]
    publisherlist.append(publisher)

In [42]:
import pandas as pd

df = pd.DataFrame(titlelist, columns = ['Title'])

In [43]:
df['Author'] = authorlist
df['Publisher'] = publisherlist
df['Price'] = pricelist

In [44]:
df.head()

,Title,Author,Publisher,Price
0,흔한남매 2,흔한남매 원저/백난도 글/유난희 그림/샌드박스네트워크 감수,미래엔아이세움,"9,900"
1,당신은 뇌를 고칠 수 있다,톰 오브라이언 저/이시은 역,브론스테인,"17,820"
2,90년생이 온다 (YES24 단독 스페셜 에디션),임홍택 저,웨일북,"12,600"
3,설민석의 삼국지 2,설민석 저,세계사,"19,800"
4,설민석의 삼국지 1,설민석 저,세계사,"19,800"
